## Données pour effectuer l'entrainement

In [1]:
import numpy as np 

fleurs_training = np.genfromtxt("fleurs_training.csv", delimiter=",")
nRow, nCol = fleurs_training.shape
print(f"There are {nRow} rows and {nCol} columns")

FileNotFoundError: fleurs_training.csv not found.

## Données pour faire le test

In [ ]:
fleurs_test = np.genfromtxt("fleurs_test.csv", delimiter=",")
nRow, nCol = fleurs_test.shape
print(f"There are {nRow} rows and {nCol} columns")

# Exercice

Voir le sujet donné sur papier

On récupère les données et leurs labels qui se trouvent dans la colonne d'indice 4

In [2]:
train_data = fleurs_training[:,0:4]
train_data.shape

NameError: name 'fleurs_training' is not defined

In [3]:
train_labels = fleurs_training[:,4]
train_labels.shape

NameError: name 'fleurs_training' is not defined

In [4]:
test_data = fleurs_test[:,0:4]
test_data.shape

NameError: name 'fleurs_test' is not defined

In [5]:
test_labels = fleurs_test[:,4]
test_labels.shape

NameError: name 'fleurs_test' is not defined

On normalise les données

In [6]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

NameError: name 'train_data' is not defined

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(3, activation="softmax")
    ])
    model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 200
all_loss_histories = []
all_val_loss_histories = []
all_acc_histories = []
all_val_acc_histories = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_labels = np.concatenate(
        [train_labels[:i * num_val_samples],
         train_labels[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_labels,
                        validation_data=(val_data, val_labels),
                        epochs=num_epochs, batch_size=32, verbose=0)
    all_loss_histories.append(history.history["loss"])
    all_val_loss_histories.append(history.history["val_loss"])
    all_acc_histories.append(history.history["accuracy"])
    all_val_acc_histories.append(history.history["val_accuracy"])

In [ ]:
average_loss = [
    np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
average_val_loss = [
    np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]
average_acc = [
    np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]
average_val_acc = [
    np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]

In [ ]:
model.summary()

**Plotting the training and validation loss**

In [ ]:
import matplotlib.pyplot as plt
epochs = range(1, len(average_loss) + 1)
plt.plot(epochs, average_loss, "r.", label="Training loss")
plt.plot(epochs, average_val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
debut_epochs = 25
epochs_restriction = range(debut_epochs+1, len(average_loss) + 1)
plt.plot(epochs_restriction, average_loss[debut_epochs:], "r.", label="Training loss")
plt.plot(epochs_restriction, average_val_loss[debut_epochs:], "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(epochs, average_acc, "r.", label="Training acc")
plt.plot(epochs, average_val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(epochs_restriction, average_acc[debut_epochs:], "r.", label="Training acc")
plt.plot(epochs_restriction, average_val_acc[debut_epochs:], "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_labels, 
          epochs=80, batch_size=32, verbose=0)

In [ ]:
predictions = model(test_data)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

**Nouvelle fleur**

In [ ]:
nouvelle_fleur = np.array([[6.6, 3.3, 5.8, 2.5]])

In [ ]:
nouvelle_fleur -= mean
nouvelle_fleur /= std

In [ ]:
prediction = model(nouvelle_fleur)
prediction = prediction.numpy()
prediction

In [ ]:
predicted_label = np.argmax(prediction, axis=1)
predicted_label